# Palisades and Eaton Fires

## About the Data





In [6]:
# Import Libraries

import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr   
import rioxarray as rioxr
import contextily as ctx

In [8]:
# Import Data 

# Fire Perimeter Data
eaton = gpd.read_file(os.path.join('data','Eaton_Perimeter_20250121','Eaton_Perimeter_20250121.shp')) # Eaton Fire Perimeter

palisades = gpd.read_file(os.path.join('data','Palisades_Perimeter_20250121','Palisades_Perimeter_20250121.shp')) # Palisades Fire Perimeter

# Landsat NetCDF File
landsat = xr.open_dataset(os.path.join('data','landsat8-2025-02-23-palisades-eaton.nc'))

## Landsat Data Exploration

In [9]:
# Explore landsat data (see attributes, variables, and dimensions)
landsat

<xarray.Dataset> Size: 78MB
Dimensions:      (y: 1418, x: 2742)
Coordinates:
  * y            (y) float64 11kB 3.799e+06 3.799e+06 ... 3.757e+06 3.757e+06
  * x            (x) float64 22kB 3.344e+05 3.344e+05 ... 4.166e+05 4.166e+05
    time         datetime64[ns] 8B ...
Data variables:
    red          (y, x) float32 16MB ...
    green        (y, x) float32 16MB ...
    blue         (y, x) float32 16MB ...
    nir08        (y, x) float32 16MB ...
    swir22       (y, x) float32 16MB ...
    spatial_ref  int64 8B ...

In [23]:
# Check units of landsat data coordinates 
print(f'x coordinates have unit: {landsat["x"].attrs.get("units")}')
print(f'y coordinates have unit: {landsat["y"].attrs.get("units")}')

x coordinates have unit: metre
y coordinates have unit: metre


## Landsat Data Exploration Summary WRITE DESCRIPTION HERE 
The `coordinates` have an `x` and `y`, signaling that the `landsat` data is projected in a **UTM projected coordinate system**

## Restore Landsat Geospatial Information

In [41]:
# Check dataset's CRS
print(landsat.rio.crs)

None


While we cannot access a `CRS` in `Landsat`, that does not mean the dataset is **not** a geospatial object. Access the `spatial_ref.crs_wkt` attribute of the dataset and print the `CRS`.

In [68]:
# Access and extract spatial_ref.crs_wkt attribute and store in variable
landsat_crs = landsat.spatial_ref.crs_wkt

type(landsat_crs) # Check that it is an acceptable object for next step

str

In [65]:
# Use rio.write_crs() to recover the geospatial information 
landsat.rio.write_crs(landsat_crs, inplace=True)

<xarray.Dataset> Size: 78MB
Dimensions:      (y: 1418, x: 2742)
Coordinates:
  * y            (y) float64 11kB 3.799e+06 3.799e+06 ... 3.757e+06 3.757e+06
  * x            (x) float64 22kB 3.344e+05 3.344e+05 ... 4.166e+05 4.166e+05
    time         datetime64[ns] 8B ...
    spatial_ref  int64 8B 0
Data variables:
    red          (y, x) float32 16MB ...
    green        (y, x) float32 16MB ...
    blue         (y, x) float32 16MB ...
    nir08        (y, x) float32 16MB ...
    swir22       (y, x) float32 16MB ...

In [69]:
# Check that dataset has been updated with the CRS
print(landsat.rio.crs)

EPSG:32611


## Fire Perimeter Data Exploration

Explore the fire perimeter data and write a brief summary of the information you obtained from the preliminary exploration

In [32]:
# Check Eaton fire perimeter data's CRS 
print(eaton.crs)

# Verify that Eaton fire perimeter data is on same CRS as the Palisades fire perimeter data
assert(eaton.crs==palisades.crs) # Assert test confirms that they are on the same CRS system (EPSG 3857)

EPSG:3857


In [29]:
# Check Eaton dataset's columns and their data types
eaton.dtypes

OBJECTID         int64
type            object
Shape__Are     float64
Shape__Len     float64
geometry      geometry
dtype: object

In [30]:
# Check Palisades dataset's columns and their data types
palisades.dtypes

OBJECTID         int64
type            object
Shape__Are     float64
Shape__Len     float64
geometry      geometry
dtype: object

## Fire Perimeter Data Exploration Summary
